In [24]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from scipy.stats import pearsonr

In [4]:
def get_X_y_math_verbal(conn_mat_filename: str, general_scores_filename: str, math_scores_filename: str, verbal_scores_filename: str):
    Glasser_conn_mat = np.load(conn_mat_filename)

    # get upper triangle indices, without diagonal 0 (k = 1 starts from the k' diagonal)
    indices = np.triu_indices(360, k=1)

    # create X: rows for subjects, and flattened upper triangle mat for each subject
    X = []
    for i in range(Glasser_conn_mat.shape[2]):
        X.append(Glasser_conn_mat[:, :, i][indices])
    # convert X to data frame for pipeline parameters
    X = pd.DataFrame(X)

    # read scores
    y = pd.read_csv(general_scores_filename, header=None).to_numpy()
    y_math = pd.read_csv(math_scores_filename, header=None).to_numpy()
    y_verbal = pd.read_csv(verbal_scores_filename, header=None).to_numpy()

    return X, y, y_math, y_verbal

In [6]:
X, y, y_math, y_verbal= get_X_y_math_verbal(
    conn_mat_filename="data/Glasser_conn_mat_158_subj.npy",
    general_scores_filename="data/general_scores_158_subj.csv",
    math_scores_filename="data/math_scores_158_subj.csv",
    verbal_scores_filename="data/verbal_scores_158_subj.csv",
)

In [7]:
X

,0,1,2,3,4,5,6,7,8,9,...,64610,64611,64612,64613,64614,64615,64616,64617,64618,64619
0,0.256043,0.543582,0.834341,0.737259,0.611785,0.551843,0.268657,0.506335,0.347711,0.313753,...,0.048507,-0.001441,0.037499,0.057737,0.108180,0.056225,0.042079,0.084301,0.041814,0.307135
1,0.483221,0.612067,0.912712,0.845826,0.808821,0.462722,0.192100,0.297347,0.760023,0.656678,...,0.455484,0.171413,0.093261,0.365236,0.236431,0.193829,0.247808,0.266242,0.295980,0.532271
2,0.126272,0.541725,0.894314,0.881399,0.768822,0.541432,0.282807,0.175312,0.360123,0.392476,...,0.291799,0.033146,0.026356,0.229688,0.116440,0.049035,0.071814,0.219577,0.078079,0.439387
3,0.528645,0.548732,0.897949,0.874365,0.722720,0.452277,0.589948,0.519460,0.393255,0.331269,...,0.277162,0.061410,0.055927,0.003119,0.063941,0.240621,0.062928,0.136225,0.082756,0.281293
4,0.408504,0.533769,0.918164,0.885054,0.722196,0.653343,0.433824,-0.006783,0.548261,0.341645,...,0.194007,-0.084169,-0.293965,-0.067129,0.015106,-0.066016,0.092298,0.061617,0.007905,0.311022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,0.497834,0.592031,0.945841,0.886041,0.737326,0.665670,0.324835,0.187993,0.534930,0.465906,...,0.435125,0.082796,0.125401,0.225401,0.045384,0.137809,0.239056,0.178493,0.124486,0.334965
154,0.274340,0.601339,0.899841,0.832969,0.679898,0.506748,0.209305,0.273583,0.391672,0.145301,...,0.274637,0.080526,0.021162,0.204128,0.080962,0.033422,0.177067,0.244266,0.238470,0.499744
155,0.649097,0.548860,0.792040,0.734714,0.501993,0.564024,0.376806,0.250711,0.529989,0.315082,...,0.481008,0.202375,-0.087569,-0.172030,0.112162,0.011920,-0.065025,-0.027739,0.059129,0.414311
156,0.457167,0.697228,0.949361,0.923404,0.861765,0.720488,0.334928,0.192592,0.507035,0.459623,...,0.261736,0.200122,0.139826,0.057603,0.277383,0.374788,0.009510,0.280210,0.196362,0.292409


In [8]:
y

array([[698.],
       [623.],
       [664.],
       [776.],
       [663.],
       [620.],
       [698.],
       [770.],
       [729.],
       [732.],
       [750.],
       [600.],
       [730.],
       [667.],
       [595.],
       [697.],
       [672.],
       [699.],
       [728.],
       [581.],
       [689.],
       [650.],
       [729.],
       [726.],
       [742.],
       [695.],
       [741.],
       [695.],
       [577.],
       [600.],
       [664.],
       [712.],
       [729.],
       [734.],
       [745.],
       [647.],
       [709.],
       [670.],
       [747.],
       [671.],
       [596.],
       [738.],
       [786.],
       [706.],
       [689.],
       [670.],
       [780.],
       [698.],
       [663.],
       [734.],
       [667.],
       [723.],
       [768.],
       [750.],
       [740.],
       [738.],
       [734.],
       [729.],
       [745.],
       [585.],
       [692.],
       [669.],
       [737.],
       [711.],
       [730.],
       [728.],
       [64

In [9]:
y_math

array([[149.],
       [133.],
       [134.],
       [143.],
       [108.],
       [110.],
       [145.],
       [150.],
       [135.],
       [140.],
       [142.],
       [120.],
       [141.],
       [137.],
       [106.],
       [138.],
       [113.],
       [138.],
       [148.],
       [117.],
       [145.],
       [113.],
       [139.],
       [130.],
       [144.],
       [140.],
       [138.],
       [145.],
       [117.],
       [ 90.],
       [110.],
       [129.],
       [139.],
       [130.],
       [145.],
       [112.],
       [138.],
       [132.],
       [136.],
       [124.],
       [110.],
       [150.],
       [148.],
       [138.],
       [148.],
       [148.],
       [150.],
       [147.],
       [132.],
       [145.],
       [130.],
       [137.],
       [148.],
       [142.],
       [149.],
       [140.],
       [138.],
       [141.],
       [135.],
       [117.],
       [136.],
       [125.],
       [146.],
       [137.],
       [150.],
       [132.],
       [ 9

In [10]:
y_verbal

array([[124.],
       [121.],
       [120.],
       [148.],
       [142.],
       [100.],
       [130.],
       [135.],
       [130.],
       [148.],
       [143.],
       [111.],
       [137.],
       [148.],
       [122.],
       [129.],
       [145.],
       [130.],
       [132.],
       [105.],
       [130.],
       [132.],
       [134.],
       [150.],
       [136.],
       [120.],
       [148.],
       [116.],
       [113.],
       [134.],
       [120.],
       [145.],
       [148.],
       [145.],
       [142.],
       [129.],
       [133.],
       [111.],
       [148.],
       [131.],
       [117.],
       [147.],
       [150.],
       [130.],
       [125.],
       [125.],
       [148.],
       [134.],
       [130.],
       [132.],
       [120.],
       [144.],
       [147.],
       [140.],
       [140.],
       [140.],
       [146.],
       [135.],
       [145.],
       [116.],
       [130.],
       [134.],
       [137.],
       [139.],
       [145.],
       [148.],
       [14

# Linear Regression - General Scores (for comparison)

In [25]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", lr.score(X_train_pca, y_train))

y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation score:  [-1.2818141  -0.66885296 -0.68311795 -0.76021504 -1.8059709  -1.55481273
 -1.15783791 -0.61605366 -1.38966716 -0.51998405]
R^2 Train score:  0.20795173256329347
Test MSE: 1.47, r: 0.052, p value: 0.85


# Linear Regression - Math Scores

In [26]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_math, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", lr.score(X_train_pca, y_train))

y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation score:  [-0.58263797 -0.74666216 -1.8529241  -0.68984266 -1.55708029 -1.57655135
 -1.46703594 -0.67100396 -1.52061832 -0.65374476]
R^2 Train score:  0.23422633906679868
Test MSE: 1.009, r: 0.203, p value: 0.45


# Linear Regression - Verbal Scores

In [27]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_verbal, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = LinearRegression()
lr.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", lr.score(X_train_pca, y_train))

y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation score:  [-1.46800768 -1.14368137 -1.36455553 -0.99748263 -1.62702485 -0.7226486
 -1.27306569 -0.67546218 -0.8628129  -0.78450683]
R^2 Train score:  0.15972297734021657
Test MSE: 1.146, r: -0.011, p value: 0.969


# Ridge - General scores for comparison

In [28]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

rdg = Ridge()
rdg.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(rdg, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", rdg.score(X_train_pca, y_train))

y_test_predicted = rdg.predict(X_test_pca)
test_mse_rdg = mean_squared_error(y_test, y_test_predicted)
r_rdg, p_val_rdg = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_rdg, 3)}, r: {np.round(r_rdg, 3)}, p value: {np.round(p_val_rdg, 3)}")

cross validation score:  [-1.28180944 -0.66885238 -0.68312027 -0.76021391 -1.80595279 -1.55480942
 -1.15783356 -0.6160493  -1.38966296 -0.51998159]
R^2 Train score:  0.20795173254783617
Test MSE: 1.47, r: 0.052, p value: 0.85


# Ridge - Math scores

In [29]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_math, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

rdg = Ridge()
rdg.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(rdg, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", rdg.score(X_train_pca, y_train))

y_test_predicted = rdg.predict(X_test_pca)
test_mse_rdg = mean_squared_error(y_test, y_test_predicted)
r_rdg, p_val_rdg = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_rdg, 3)}, r: {np.round(r_rdg, 3)}, p value: {np.round(p_val_rdg, 3)}")

cross validation score:  [-0.58263711 -0.74666334 -1.85291591 -0.68983986 -1.55705008 -1.57653893
 -1.46702418 -0.6709899  -1.52061445 -0.65374161]
R^2 Train score:  0.23422633904858192
Test MSE: 1.009, r: 0.203, p value: 0.45


# Ridge - verbal scores

In [30]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_verbal, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

rdg = Ridge()
rdg.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(rdg, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", rdg.score(X_train_pca, y_train))

y_test_predicted = rdg.predict(X_test_pca)
test_mse_rdg = mean_squared_error(y_test, y_test_predicted)
r_rdg, p_val_rdg = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_rdg, 3)}, r: {np.round(r_rdg, 3)}, p value: {np.round(p_val_rdg, 3)}")

cross validation score:  [-1.46800586 -1.14367927 -1.36455801 -0.99748039 -1.62701703 -0.72264559
 -1.27306207 -0.67546051 -0.86280561 -0.7845033 ]
R^2 Train score:  0.15972297733059382
Test MSE: 1.146, r: -0.011, p value: 0.969


# Lasso - general scores for comparison

In [31]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = Lasso()
lr.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", lr.score(X_train_pca, y_train))

y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation score:  [-1.19800335 -0.7225765  -0.78912827 -0.78396773 -1.6252855  -1.59285778
 -1.10947967 -0.57467225 -1.34629703 -0.47906367]
R^2 Train score:  0.18565988431273794
Test MSE: 1.399, r: 0.191, p value: 0.479


# Lasso - Math scores

In [32]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_math, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = Lasso()
lr.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", lr.score(X_train_pca, y_train))

y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation score:  [-0.57239194 -0.77684478 -1.56965015 -0.68293803 -1.2937692  -1.35428023
 -1.30296963 -0.52204149 -1.46662509 -0.59886447]
R^2 Train score:  0.21322538272539815
Test MSE: 0.991, r: 0.202, p value: 0.452


# Lasso - verbal scores

In [33]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_verbal, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = Lasso()
lr.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", lr.score(X_train_pca, y_train))

y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation score:  [-1.42978978 -1.12452779 -1.51747579 -0.92974648 -1.51098359 -0.69585245
 -1.2300397  -0.67009528 -0.75607143 -0.68348459]
R^2 Train score:  0.13864232204897353
Test MSE: 1.072, r: 0.072, p value: 0.79


# Elastic net - general scores for comparison

In [34]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = ElasticNet()
lr.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", lr.score(X_train_pca, y_train))

y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation score:  [-1.22209041 -0.69626841 -0.72218165 -0.77647268 -1.67728205 -1.56009265
 -1.11011769 -0.58946189 -1.35776575 -0.49884928]
R^2 Train score:  0.20187697135245597
Test MSE: 1.428, r: 0.127, p value: 0.639


# Elastic net - math scores

In [35]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_math, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = ElasticNet()
lr.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", lr.score(X_train_pca, y_train))

y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation score:  [-0.57941432 -0.75198563 -1.7009691  -0.68987898 -1.35573364 -1.43736374
 -1.37803323 -0.55055475 -1.48920863 -0.6132642 ]
R^2 Train score:  0.22840012220272354
Test MSE: 1.003, r: 0.195, p value: 0.469


# Elastic net - verbal scores

In [36]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_verbal, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = ElasticNet()
lr.fit(X_train_pca, y_train)

cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation score: ", cross_val_score_list)
print("R^2 Train score: ", lr.score(X_train_pca, y_train))

y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation score:  [-1.44590021 -1.13571277 -1.46183202 -0.9584726  -1.55976094 -0.70858743
 -1.26508302 -0.68000573 -0.80108539 -0.73866884]
R^2 Train score:  0.15381780746615548
Test MSE: 1.11, r: 0.021, p value: 0.939


# Random Forest - general scores for comparison

In [38]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=75, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

# Parameters grid for hyperparameter tuning 
param_grid_rf = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 5, 10],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'max_features': ['sqrt', 'log2'],  # Number of features to consider when looking for the best split
}

r_vec = []
p_value_vec = []
fold_mse_train = []
fold_mse_val = []
fold_config = []

kf = KFold(n_splits=10, shuffle=True, random_state=0)

for train_index, val_index in kf.split(X_train_pca):
    X_train_fold, X_val_fold = X_train_pca[train_index], X_train_pca[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    random_forest = RandomForestRegressor(random_state=0)
    grid_search = GridSearchCV(
        random_forest,
        param_grid=param_grid_rf,
        scoring="neg_mean_squared_error",
        cv=10,
        n_jobs=-1,
    )
    grid_search.fit(X_train_fold, y_train_fold)

    # Fit and Predict
    best_estimator = grid_search.best_estimator_
    y_train_predicted = best_estimator.predict(X_train_fold)
    y_predicted = best_estimator.predict(X_val_fold)
    fold_config.append(grid_search.best_params_)
    train_mse, val_mse = mean_squared_error(y_train_fold, y_train_predicted), mean_squared_error(y_val_fold, y_predicted)

    r, p_value = pearsonr(y_predicted, y_val_fold)
    r_vec.append(r)
    p_value_vec.append(p_value)
    fold_mse_train.append(train_mse)
    fold_mse_val.append(val_mse)

print("mean r: ", np.mean(r_vec), ", mean p-value: ", np.mean(p_value_vec),  ", mean val MSE: ", np.mean(fold_mse_val))
print(f"min MSE: {np.min(fold_mse_val)} in fold {np.argmin(fold_mse_val) + 1}")
# Configuration that received the minimal MSE
config = fold_config[np.argmin(fold_mse_val)]
print(config)

# testing model on test set
chosen_random_forest = RandomForestRegressor(random_state=0, **config)
chosen_random_forest.fit(X_train_pca, y_train)

y_pred = chosen_random_forest.predict(X_test_pca)
mse = mean_squared_error(y_test, y_pred)
r, p_value = pearsonr(y_test, y_pred)
print(f"MSE: {np.round(mse, 3)}, r: {np.round(r, 3)}, p value: {np.round(p_value, 3)}")


mean r:  0.1296659421932517 , mean p-value:  0.4914199451208222 , mean val MSE:  1.0149050865181246
min MSE: 0.579972475340763 in fold 4
{'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
MSE: 1.389, r: 0.235, p value: 0.381


# Random Forest - math scores

In [39]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_math, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=75, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

# Parameters grid for hyperparameter tuning 
param_grid_rf = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 5, 10],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'max_features': ['sqrt', 'log2'],  # Number of features to consider when looking for the best split
}

r_vec = []
p_value_vec = []
fold_mse_train = []
fold_mse_val = []
fold_config = []

kf = KFold(n_splits=10, shuffle=True, random_state=0)

for train_index, val_index in kf.split(X_train_pca):
    X_train_fold, X_val_fold = X_train_pca[train_index], X_train_pca[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    random_forest = RandomForestRegressor(random_state=0)
    grid_search = GridSearchCV(
        random_forest,
        param_grid=param_grid_rf,
        scoring="neg_mean_squared_error",
        cv=10,
        n_jobs=-1,
    )
    grid_search.fit(X_train_fold, y_train_fold)

    # Fit and Predict
    best_estimator = grid_search.best_estimator_
    y_train_predicted = best_estimator.predict(X_train_fold)
    y_predicted = best_estimator.predict(X_val_fold)
    fold_config.append(grid_search.best_params_)
    train_mse, val_mse = mean_squared_error(y_train_fold, y_train_predicted), mean_squared_error(y_val_fold, y_predicted)

    r, p_value = pearsonr(y_predicted, y_val_fold)
    r_vec.append(r)
    p_value_vec.append(p_value)
    fold_mse_train.append(train_mse)
    fold_mse_val.append(val_mse)

print("mean r: ", np.mean(r_vec), ", mean p-value: ", np.mean(p_value_vec),  ", mean val MSE: ", np.mean(fold_mse_val))
print(f"min MSE: {np.min(fold_mse_val)} in fold {np.argmin(fold_mse_val) + 1}")
# Configuration that received the minimal MSE
config = fold_config[np.argmin(fold_mse_val)]
print(config)

# testing model on test set
chosen_random_forest = RandomForestRegressor(random_state=0, **config)
chosen_random_forest.fit(X_train_pca, y_train)

y_pred = chosen_random_forest.predict(X_test_pca)
mse = mean_squared_error(y_test, y_pred)
r, p_value = pearsonr(y_test, y_pred)
print(f"MSE: {np.round(mse, 3)}, r: {np.round(r, 3)}, p value: {np.round(p_value, 3)}")


mean r:  0.17857560160950278 , mean p-value:  0.5083428144920112 , mean val MSE:  0.991687101466925
min MSE: 0.38100181069770295 in fold 5
{'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
MSE: 0.853, r: 0.66, p value: 0.005


# Random Forest - verbal scores

In [40]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_verbal, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=75, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

# Parameters grid for hyperparameter tuning 
param_grid_rf = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 5, 10],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'max_features': ['sqrt', 'log2'],  # Number of features to consider when looking for the best split
}

r_vec = []
p_value_vec = []
fold_mse_train = []
fold_mse_val = []
fold_config = []

kf = KFold(n_splits=10, shuffle=True, random_state=0)

for train_index, val_index in kf.split(X_train_pca):
    X_train_fold, X_val_fold = X_train_pca[train_index], X_train_pca[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    random_forest = RandomForestRegressor(random_state=0)
    grid_search = GridSearchCV(
        random_forest,
        param_grid=param_grid_rf,
        scoring="neg_mean_squared_error",
        cv=10,
        n_jobs=-1,
    )
    grid_search.fit(X_train_fold, y_train_fold)

    # Fit and Predict
    best_estimator = grid_search.best_estimator_
    y_train_predicted = best_estimator.predict(X_train_fold)
    y_predicted = best_estimator.predict(X_val_fold)
    fold_config.append(grid_search.best_params_)
    train_mse, val_mse = mean_squared_error(y_train_fold, y_train_predicted), mean_squared_error(y_val_fold, y_predicted)

    r, p_value = pearsonr(y_predicted, y_val_fold)
    r_vec.append(r)
    p_value_vec.append(p_value)
    fold_mse_train.append(train_mse)
    fold_mse_val.append(val_mse)

print("mean r: ", np.mean(r_vec), ", mean p-value: ", np.mean(p_value_vec),  ", mean val MSE: ", np.mean(fold_mse_val))
print(f"min MSE: {np.min(fold_mse_val)} in fold {np.argmin(fold_mse_val) + 1}")
# Configuration that received the minimal MSE
config = fold_config[np.argmin(fold_mse_val)]
print(config)

# testing model on test set
chosen_random_forest = RandomForestRegressor(random_state=0, **config)
chosen_random_forest.fit(X_train_pca, y_train)

y_pred = chosen_random_forest.predict(X_test_pca)
mse = mean_squared_error(y_test, y_pred)
r, p_value = pearsonr(y_test, y_pred)
print(f"MSE: {np.round(mse, 3)}, r: {np.round(r, 3)}, p value: {np.round(p_value, 3)}")


mean r:  0.0893067753039939 , mean p-value:  0.5900633083821171 , mean val MSE:  1.03316644759123
min MSE: 0.47022371930182943 in fold 5
{'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
MSE: 1.054, r: 0.33, p value: 0.212


# Gradient boosting - general scores for comparison

In [41]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

gbr = GradientBoostingRegressor(random_state=0)
gbr.fit(X_train_pca, y_train)

y_test_predicted = gbr.predict(X_test_pca)
test_mse_gbr = mean_squared_error(y_test, y_test_predicted)
r_gbr, p_val_gbr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_gbr, 3)}, r: {np.round(r_gbr, 3)}, p value: {np.round(p_val_gbr, 3)}")

Test MSE: 1.358, r: 0.299, p value: 0.26


# Gradient Boosting - math scores

In [42]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_math, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

gbr = GradientBoostingRegressor(random_state=0)
gbr.fit(X_train_pca, y_train)

y_test_predicted = gbr.predict(X_test_pca)
test_mse_gbr = mean_squared_error(y_test, y_test_predicted)
r_gbr, p_val_gbr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_gbr, 3)}, r: {np.round(r_gbr, 3)}, p value: {np.round(p_val_gbr, 3)}")

Test MSE: 0.903, r: 0.309, p value: 0.244


# Gradient boosting - verbal scores

In [45]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_verbal, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

gbr = GradientBoostingRegressor(random_state=0)
gbr.fit(X_train_pca, y_train)

y_test_predicted = gbr.predict(X_test_pca)
test_mse_gbr = mean_squared_error(y_test, y_test_predicted)
r_gbr, p_val_gbr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_gbr, 3)}, r: {np.round(r_gbr, 3)}, p value: {np.round(p_val_gbr, 3)}")

Test MSE: 1.262, r: -0.203, p value: 0.451
